In [9]:
# generate_project4_outputs.py
import os
import pandas as pd
import matplotlib.pyplot as plt

DATA_PATH = "/content/retail_sales_project.csv"   # change if needed
OUT_DIR = "project4_output"
PLOTS_DIR = os.path.join(OUT_DIR, "plots")
os.makedirs(PLOTS_DIR, exist_ok=True)

# 1) Load data
df = pd.read_csv(DATA_PATH, parse_dates=['date'])
# quick checks
print(df.head(10))
print(df.info())
print("Missing values per column:\n", df.isna().sum())
print("Date range:", df['date'].min(), "to", df['date'].max())

# 2) Create derived time columns
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['month_name'] = df['date'].dt.to_period('M').astype(str)
df['week'] = df['date'].dt.isocalendar().week

# 3) Aggregations
daily_sales = df.groupby('date')['total_sales'].sum().reset_index(name='daily_sales')
monthly_sales = df.groupby('month_name')['total_sales'].sum().reset_index(name='monthly_sales')
product_sales = df.groupby(['product_id','category'])[['total_sales','units_sold']].sum().reset_index()
product_sales = product_sales.sort_values('total_sales', ascending=False)

# 4) Plot - daily revenue trend
plt.figure(figsize=(10,4))
plt.plot(daily_sales['date'], daily_sales['daily_sales'])
plt.title('Daily Revenue Trend')
plt.xlabel('Date')
plt.ylabel('Revenue')
plt.tight_layout()
plt.savefig(os.path.join(PLOTS_DIR, "daily_revenue_trend.png"))
plt.close()

# 5) Plot - monthly revenue bar
plt.figure(figsize=(10,4))
plt.bar(monthly_sales['month_name'], monthly_sales['monthly_sales'])
plt.title('Monthly Revenue')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig(os.path.join(PLOTS_DIR, "monthly_revenue_bar.png"))
plt.close()

# 6) Plot - top products by revenue (top 6)
top_products = product_sales.head(6)
plt.figure(figsize=(8,4))
plt.barh(top_products['product_id'][::-1], top_products['total_sales'][::-1])
plt.title('Top Products by Revenue (top 6)')
plt.tight_layout()
plt.savefig(os.path.join(PLOTS_DIR, "top_products_revenue.png"))
plt.close()

# 7) Plot - promo effect (boxplot)
plt.figure(figsize=(6,4))
data_no = df[df['promotion']==0]['total_sales']
data_yes = df[df['promotion']==1]['total_sales']
plt.boxplot([data_no, data_yes], tick_labels=['No Promo','Promo'])
plt.title('Sales distribution: Promo vs No Promo')
plt.tight_layout()
plt.savefig(os.path.join(PLOTS_DIR, "promo_vs_no_promo_boxplot.png"))
plt.close()

# 8) Rolling mean and plot (use small window because your series is 15-day freq)
daily_sales['ma_3'] = daily_sales['daily_sales'].rolling(window=3, min_periods=1).mean()
plt.figure(figsize=(10,4))
plt.plot(daily_sales['date'], daily_sales['daily_sales'], label='Daily Sales')
plt.plot(daily_sales['date'], daily_sales['ma_3'], label='Rolling Mean (3)')
plt.legend()
plt.title('Daily Sales with Rolling Mean')
plt.tight_layout()
plt.savefig(os.path.join(PLOTS_DIR, "daily_with_rolling_mean.png"))
plt.close()

# 9) Save aggregation tables (optional)
daily_sales.to_csv(os.path.join(OUT_DIR, "daily_sales_aggregated.csv"), index=False)
monthly_sales.to_csv(os.path.join(OUT_DIR, "monthly_sales_aggregated.csv"), index=False)
product_sales.to_csv(os.path.join(OUT_DIR, "product_sales_aggregated.csv"), index=False)

# 10) Naive forecast (last monthly total)
last_month = monthly_sales.iloc[-1]
naive_forecast_value = float(last_month['monthly_sales'])
naive_forecast = {"forecast_month": "next_month", "forecast_value": naive_forecast_value}
print("Naive forecast (next_month):", naive_forecast_value)



        date store_id product_id     category  units_sold  unit_price  \
0 2022-01-01     S001       P001  Electronics          97         399   
1 2022-01-01     S001       P002     Clothing          59          49   
2 2022-01-01     S001       P003    Groceries          63           5   
3 2022-01-01     S002       P001  Electronics         107         399   
4 2022-01-01     S002       P002     Clothing          77          49   
5 2022-01-01     S002       P003    Groceries          66           5   
6 2022-01-01     S003       P001  Electronics          38         399   
7 2022-01-01     S003       P002     Clothing         137          49   
8 2022-01-01     S003       P003    Groceries         143           5   
9 2022-01-16     S001       P001  Electronics         112         399   

   total_sales  promotion region  
0        38703          0  North  
1         2891          0  North  
2          315          0  North  
3        42693          1  South  
4         3773       